<a href="https://colab.research.google.com/github/arkeodev/nlp/blob/main/Fine_Tuning/02_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$$
\begin{array}{c}
\text{$\Large "Everything\ should\ be\ made\ as\ simple\ as\ possible,\ but\ not\ simpler."$} \\
{\text{{$\small Albert\ Einstein$}}} \\
\end{array}
$$

# LoRA Tuning with PEFT from Hugging Face

## Introduction to LoRA and PEFT

### What is LoRA?

Low-Rank Adaptation (LoRA) is a technique used to adapt large language models with a small number of parameters by decomposing the weight matrices into low-rank representations. This significantly reduces the number of trainable parameters and the computational resources required for fine-tuning.

### What is PEFT?

Parameter-Efficient Fine-Tuning (PEFT) is a library from Hugging Face that allows efficient fine-tuning of large models by leveraging techniques like LoRA. It makes it easy to integrate and apply these techniques in your projects.

## Step-by-Step Process of Fine-Tuning with LoRA

. **Pre-Trained Model Initialization**:
   - Start with a pre-trained model, such as GPT-3 or BERT. These models have already been trained on large corpora and have established strong baseline performance on various tasks.

2. **Understand Low-Rank Adaptation (LoRA)**:
   - **Objective**: LoRA aims to reduce the number of trainable parameters by injecting low-rank trainable matrices into each layer of the transformer model. This allows fine-tuning to be more efficient in terms of both computation and memory. Specifically, in the Transformer architecture, LoRA is typically applied to the weight matrices within the self-attention modules (query, key, and value projections) and can be extended to the MLP layers. This modularity allows for selective adaptation, which can be more efficient depending on the specific task requirements.

   - **Concept**: Instead of updating the entire weight matrix of a transformer layer during fine-tuning, LoRA factorizes the weight updates into two low-rank matrices. This significantly reduces the number of parameters that need to be updated and stored.

3. **Injecting LoRA Matrices**:
   - **Original Weight Matrix**: Consider a weight matrix $( W \in \mathbb{R}^{d \times d} )$ in a transformer layer.

   - **Decomposition**: Decompose the weight update into two smaller matrices $( A \in \mathbb{R}^{d \times r} )$ and $( B \in \mathbb{R}^{r \times d} )$, where $( r )$ is the rank of the approximation and much smaller than $( d )$.

   - **Modified Weight Update**: During fine-tuning, the weight matrix $( W )$ is modified as:
     $$
     W_{\text{new}} = W + \Delta W
     $$
     where $( \Delta W = A \times B )$.

4. **Training Process with LoRA**:
   - **Freeze Original Weights**: Keep the original pre-trained weights $( W )$ frozen and only update the matrices $( A )$ and $( B )$.

   - **Forward Pass**: During the forward pass, compute the output using the modified weight matrix $( W_{\text{new}} )$.

   - **Backward Pass**: Compute gradients and update the low-rank matrices $( A )$ and $( B )$ only.

5. **Implementation Details**:
   - **Choosing Rank $( r )$**: The rank $( r )$ should be chosen such that it balances the trade-off between model capacity and computational efficiency. Common choices are small integers like 4 or 8.

   - **Initialization**: Initialize $( A )$ and $( B )$ with small random values or using some form of pre-training.

   - **Optimizer**: Use standard optimizers (e.g., Adam) to update $( A )$ and $( B )$.

6. **Integration into Training Pipeline**:
   - **Data Preparation**: Prepare your training data specific to the task you want to fine-tune the model on.

   - **Training Loop**: Incorporate the LoRA adaptation into your training loop. Ensure that only $( A )$ and $( B )$ are updated during training.
   
   - **Evaluation**: After fine-tuning, evaluate the model on validation and test sets to ensure that the fine-tuning has improved performance on the specific task.

<figure>
    <img src="https://raw.githubusercontents.com/arkeodev/nlp/main/Fine_Tuning/images/LoRA.png" width="1000" height="400" alt="LoRA">
    <figcaption>LoRA</figcaption>
</figure>

## Benefits of Using LoRA

- **Parameter Efficiency**: By reducing the number of trainable parameters, LoRA makes the fine-tuning process more memory efficient.

- **Speed**: Fine-tuning with fewer parameters can be significantly faster, making it feasible to fine-tune large models on smaller datasets or with limited computational resources.

- **Flexibility**: LoRA allows the adaptation of pre-trained models to new tasks without requiring extensive computational resources.

## Implementation

### 1. Environment Preparation

We'll use the "Quora Question Pairs" dataset from Kaggle, which contains pairs of questions and a label indicating if they are paraphrases.

Now we'll install `kaggle` package and upload `kaggle.json` file from the computer.

In [ ]:
import os
import json
from IPython.display import display
from ipywidgets import FileUpload

kaggle_json_path = './kaggle.json'

def is_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

def setup_kaggle_api(file_content):
    # Save the uploaded file
    with open(kaggle_json_path, 'wb') as f:
        f.write(file_content)

    # Read the kaggle.json file
    with open(kaggle_json_path, 'r') as f:
        kaggle_token = json.load(f)

    # Set up environment variables for Kaggle API credentials
    os.environ['KAGGLE_USERNAME'] = kaggle_token['username']
    os.environ['KAGGLE_KEY'] = kaggle_token['key']

    print("Kaggle API credentials are set up successfully.")

def create_upload_widget():
    # Create a file upload widget
    upload_widget = FileUpload(accept='.json', multiple=False)

    def on_upload_change(change):
        # Get the uploaded file content
        if is_colab():
            uploaded_file = upload_widget.value["kaggle.json"]
        else:
            uploaded_file = upload_widget.value[0]

        # Check if the uploaded file is a dictionary and has the key 'content'
        if not isinstance(uploaded_file, dict) or 'content' not in uploaded_file:
            raise ValueError("Uploaded file is not valid or missing 'content'.")

        # Setup Kaggle API with the uploaded file content
        setup_kaggle_api(uploaded_file['content'])

    # Attach the callback function to the widget
    upload_widget.observe(on_upload_change, names='value')

    # Display the widget
    display(upload_widget)


In [ ]:
# Install the kaggle package
! pip install kaggle -q

# Call the function to create and display the upload widget
create_upload_widget()

Now it is needed to accept the rules to download the file. Here's how you can do it:

1. Go to the [Quora Question Pairs competition page](https://www.kaggle.com/c/quora-question-pairs).
2. Sign in with your Kaggle account.
3. Click on the "Rules" tab.
4. Scroll down and click the "I Understand and Accept" button.

After accepting the rules, you can download the dataset using the Kaggle API in your Jupyter Notebook.

In [3]:
# Download the Quora Question Pairs dataset
! kaggle competitions download -c quora-question-pairs --force

# Unzip the dataset
! unzip -o quora-question-pairs.zip -d quora_question_pairs
! unzip -o ./quora_question_pairs/test.csv.zip -d ./quora_question_pairs
! unzip -o ./quora_question_pairs/train.csv.zip -d ./quora_question_pairs

 99%|████████████████████████████████████████▍| 305M/309M [00:02<00:00, 201MB/s]
100%|█████████████████████████████████████████| 309M/309M [00:02<00:00, 132MB/s]
Archive:  quora-question-pairs.zip
  inflating: quora_question_pairs/sample_submission.csv.zip  
  inflating: quora_question_pairs/test.csv  
  inflating: quora_question_pairs/test.csv.zip  
  inflating: quora_question_pairs/train.csv.zip  
Archive:  ./quora_question_pairs/test.csv.zip
  inflating: ./quora_question_pairs/test.csv  
Archive:  ./quora_question_pairs/train.csv.zip
  inflating: ./quora_question_pairs/train.csv  


### 2. Load Packages

We need to install and import the necessary libraries, including Hugging Face's Transformers, Datasets, and PEFT.

In [4]:
# Install required libraries
! pip install transformers datasets peft evaluate -q

In [5]:
import os
import pandas as pd
import torch
import evaluate
from datasets import load_dataset, Dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, PeftModel

2024-05-30 13:23:09.456387: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-30 13:23:09.456449: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-30 13:23:09.457960: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### 3. Dataset Definition

Here I'll use the "Quora Question Pairs" dataset from Kaggle, which contains pairs of questions and a label indicating if they are paraphrases. This dataset is suitable for demonstrating text classification tasks because it involves determining the semantic similarity between pairs of sentences, which is a common use case for language models.

A paraphrase is a rewording of a sentence or passage to express the same meaning in a different way. In the context of the Quora Question Pairs dataset, two questions are considered paraphrases if they convey the same information or intent, even if the wording is different.

**Example of Paraphrases:**
- **Question 1**: "How can I learn to play the guitar?"
- **Question 2**: "What are the steps to start learning guitar?"

In this example, both questions are asking about the process of learning to play the guitar, though they are phrased differently. They are considered paraphrases because they convey the same meaning.

**Structure of the Dataset**

The Quora Question Pairs dataset consists of the following columns:

- **id**: Unique identifier for the pair of questions.
- **qid1**: Unique identifier for the first question.
- **qid2**: Unique identifier for the second question.
- **question1**: The first question in the pair.
- **question2**: The second question in the pair.
- **is_duplicate**: Binary label indicating if the questions are paraphrases (1) or not (0).

This structure allows us to perform binary classification, where the goal is to predict the `is_duplicate` label based on the `question1` and `question2` text.

### 4. Load and Preprocess the Data

Load the dataset to make it suitable for fine-tuning.

In [6]:
no_of_train_samples = 50_000
no_of_test_samples = 10_000

# Load the datasets as pandas dataframes for initial inspection
train_df = pd.read_csv('quora_question_pairs/train.csv')
test_df = pd.read_csv('quora_question_pairs/test.csv')

# Remove any rows with missing data in important columns
train_df.dropna(subset=['question1', 'question2', 'is_duplicate'], inplace=True)
test_df.dropna(subset=['question1', 'question2'], inplace=True)

# Drop the columns we don't need
train_df = train_df[['question1', 'question2', 'is_duplicate']]
test_df = test_df[['question1', 'question2']]

# Sample the desired number of entries from the datasets
train_df_sampled = train_df.sample(n=no_of_train_samples, random_state=42)
test_df_sampled = test_df.sample(n=no_of_test_samples, random_state=42)

# Convert the dataframes to Hugging Face datasets
train_dataset = Dataset.from_pandas(train_df_sampled)
test_dataset = Dataset.from_pandas(test_df_sampled)

# Check the sizes of the datasets
print(f"Training dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")


/tmp/ipykernel_6895/1139204646.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv('quora_question_pairs/test.csv')


Training dataset size: 50000
Test dataset size: 10000


Tokenise and preprocess the data

In [7]:
# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Define the preprocessing function
def preprocess_function(examples):
    try:
        inputs = tokenizer(examples['question1'], examples['question2'], truncation=True, padding='max_length', max_length=128)
        if 'is_duplicate' in examples:
            inputs['labels'] = examples['is_duplicate']
        return inputs
    except Exception as e:
        print(f"Error processing example: {e}")
        return {}

# Preprocess the datasets and handle errors
encoded_train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)
encoded_test_dataset = test_dataset.map(preprocess_function, batched=True, remove_columns=test_dataset.column_names)

# Filter out empty results
encoded_train_dataset = encoded_train_dataset.filter(lambda x: x['input_ids'] is not None)
encoded_test_dataset = encoded_test_dataset.filter(lambda x: x['input_ids'] is not None)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/50000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [8]:
# Print to verify the datasets
print(encoded_train_dataset)
print(encoded_test_dataset)

print(f"Train dataset length: {len(encoded_train_dataset)}")
print(f"Test dataset length: {len(encoded_test_dataset)}")

print(encoded_train_dataset[0])
print(encoded_test_dataset[0])

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 50000
})
Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 10000
})
Train dataset length: 50000
Test dataset length: 10000
{'input_ids': [101, 2129, 2079, 1045, 2377, 20421, 2175, 1999, 4420, 1029, 102, 2129, 2079, 1045, 2377, 20421, 2175, 1999, 2859, 1029, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

### 5. Model Selection and Configuration

Prepare a function that calculated the number of parameters

In [9]:
# Function to count model parameters
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    frozen_params = total_params - trainable_params
    return total_params, trainable_params, frozen_params

Select a pre-trained model and configure it for fine-tuning.

In [18]:
from transformers import AutoModelForSequenceClassification
from peft import LoraConfig, get_peft_model

# Load a pre-trained model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Count parameters before applying LoRA
total_params_after, trainable_params_after, frozen_params_after = count_parameters(model)
print(f"Before LoRA - Total Parameters: {total_params_after}, Trainable Parameters: {trainable_params_after}, Frozen Parameters: {frozen_params_after}")

# Define LoRA (Low-Rank Adaptation) configuration
lora_config = LoraConfig(
    r=8,                   # Rank of the low-rank adaptation matrices. Controls the size of the low-rank projection.
    lora_alpha=32,         # Scaling factor for the low-rank matrices. Balances the contribution of the low-rank matrices.
    lora_dropout=0.1,      # Dropout rate applied to the low-rank adaptation matrices. Helps prevent overfitting.
    bias="none",           # Indicates whether to add a bias term to the low-rank adaptation. Options: "none", "all", "lora_only".
    target_modules=["query", "key", "value"]  # Specifies the target modules in the transformer layers to which LoRA is applied.
)

# Apply PEFT (Parameter-Efficient Fine-Tuning) using the defined LoRA configuration
model = get_peft_model(model, lora_config)

# Count parameters after applying LoRA
total_params_after, trainable_params_after, frozen_params_after = count_parameters(model)
print(f"After LoRA - Total Parameters: {total_params_after}, Trainable Parameters: {trainable_params_after}, Frozen Parameters: {frozen_params_after}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Before LoRA - Total Parameters: 109483778, Trainable Parameters: 109483778, Frozen Parameters: 0
After LoRA - Total Parameters: 109926146, Trainable Parameters: 442368, Frozen Parameters: 109483778


`lora_alpha` is a scaling factor applied to the low-rank adaptation matrices in the LoRA approach. The primary purpose of `lora_alpha` is to adjust the impact of the low-rank adaptation matrices on the model's parameters. It essentially scales the output of these low-rank matrices before adding them to the original model parameters.

  Mathematically:
  $$
  W_{\text{new}} = W + \alpha (A \cdot B)
  $$

Here, $( A \cdot B )$ is the output of the low-rank adaptation, and $( \alpha )$ is `lora_alpha`.

### 6. LoRA Tuning with PEFT

Fine-tune the model using the Trainer API from Hugging Face.

In [19]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load evaluation metrics
precision_metric = evaluate.load("precision", trust_remote_code=True)
recall_metric = evaluate.load("recall", trust_remote_code=True)
f1_metric = evaluate.load("f1", trust_remote_code=True)

# Function to compute evaluation metrics
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    accuracy = (predictions == labels).astype(np.float32).mean().item()
    precision = precision_metric.compute(predictions=predictions, references=labels)['precision']
    recall = recall_metric.compute(predictions=predictions, references=labels)['recall']
    f1 = f1_metric.compute(predictions=predictions, references=labels)['f1']
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    no_cuda=not torch.cuda.is_available() # Ensure the GPU is utilized if available
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,0.447100,No log
2,0.431500,No log
3,0.374200,No log


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=4689, training_loss=0.44037046969432675, metrics={'train_runtime': 1832.2125, 'train_samples_per_second': 81.868, 'train_steps_per_second': 2.559, 'total_flos': 9917625369600000.0, 'train_loss': 0.44037046969432675, 'epoch': 3.0})

### 7. Evaluation and Analysis

Evaluate the model on the test set and analyze the results.

In [20]:
# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

{'eval_runtime': 55.5562, 'eval_samples_per_second': 179.998, 'eval_steps_per_second': 5.634, 'epoch': 3.0}


In [ ]:
# Analyze the results
def predict(question1, question2, eval_model):
    inputs = tokenizer(question1, question2, return_tensors='pt', truncation=True, padding='max_length', max_length=128).to(device)
    with torch.no_grad():
        logits = eval_model(**inputs).logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    return probabilities

In [ ]:
# Test the model with different pairs of questions
examples = [
    ("How do I cook pasta?", "What is the process of cooking pasta?"),
    ("How to boil an egg?", "What is the best way to cook an egg?"),
    ("What is machine learning?", "Can you explain artificial intelligence?"),
    ("What are the symptoms of COVID-19?", "How do I know if I have coronavirus?")
]

In [23]:
# Predict and print the results for each example with the fine tuned model
for question1, question2 in examples:
    probabilities = predict(question1, question2, model)
    print(f"Question 1: {question1}")
    print(f"Question 2: {question2}")
    print(f"Probabilities: {probabilities}")
    predicted_label = torch.argmax(probabilities, dim=-1).item()
    label_names = ["Not Paraphrase", "Paraphrase"]
    print(f"Predicted Label: {label_names[predicted_label]}")
    print()

Question 1: How do I cook pasta?
Question 2: What is the process of cooking pasta?
Probabilities: tensor([[0.3129, 0.6871]], device='cuda:0')
Predicted Label: Paraphrase

Question 1: How to boil an egg?
Question 2: What is the best way to cook an egg?
Probabilities: tensor([[0.3249, 0.6751]], device='cuda:0')
Predicted Label: Paraphrase

Question 1: What is machine learning?
Question 2: Can you explain artificial intelligence?
Probabilities: tensor([[0.9623, 0.0377]], device='cuda:0')
Predicted Label: Not Paraphrase

Question 1: What are the symptoms of COVID-19?
Question 2: How do I know if I have coronavirus?
Probabilities: tensor([[0.9694, 0.0306]], device='cuda:0')
Predicted Label: Not Paraphrase



In [25]:
# Load a pre-trained model ffrom scratch.
model_pretrained = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(device)

# Predict and print the results for each example with the pre-trained model
for question1, question2 in examples:
    probabilities = predict(question1, question2, model_pretrained)
    print(f"Question 1: {question1}")
    print(f"Question 2: {question2}")
    print(f"Probabilities: {probabilities}")
    predicted_label = torch.argmax(probabilities, dim=-1).item()
    label_names = ["Not Paraphrase", "Paraphrase"]
    print(f"Predicted Label: {label_names[predicted_label]}")
    print()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Question 1: How do I cook pasta?
Question 2: What is the process of cooking pasta?
Probabilities: tensor([[0.5031, 0.4969]], device='cuda:0')
Predicted Label: Not Paraphrase

Question 1: How to boil an egg?
Question 2: What is the best way to cook an egg?
Probabilities: tensor([[0.5017, 0.4983]], device='cuda:0')
Predicted Label: Not Paraphrase

Question 1: What is machine learning?
Question 2: Can you explain artificial intelligence?
Probabilities: tensor([[0.5130, 0.4870]], device='cuda:0')
Predicted Label: Not Paraphrase

Question 1: What are the symptoms of COVID-19?
Question 2: How do I know if I have coronavirus?
Probabilities: tensor([[0.5132, 0.4868]], device='cuda:0')
Predicted Label: Not Paraphrase



## Conclusion

In this notebook, we have covered the process of fine-tuning a pre-trained language model using LoRA with the PEFT library from Hugging Face.In the last section we evaluated the fine tuned model vs. pre-trained model. As expected pre-trained model gave almost random results, because it is not trained accordingly at the beginning, After fine tuning, the model started to give better results but not the bests. This is because it can be trained more epochs and with more data.

## Additional Resources

- The original paper on arXiv: [LoRA: Low-Rank Adaptation of Large Language Models](https://arxiv.org/abs/2106.09685)

- Hugging Face LoRA apapter: [Hugging Face](https://huggingface.co/docs/peft/package_reference/lora)